In [5]:
import yahoo_fin.stock_info as si
import pandas as pd
from joblib import load

multipliers = {'K':1000, 'M':1000000, 'B':1000000000, 'T':100000000000}

def string_to_int(string):
    if string[-1].isdigit():
        return int(string)
    mult = multipliers[string[-1]]
    return int(float(string[:-1]) * mult)

def run():
    # get ticker from user
    ticker = input("Enter Ticker of a company you want to check")

    # Load financial data

    financials = si.get_financials(ticker)
    quarterly_income_statement = pd.DataFrame(financials['quarterly_income_statement'])
    quarterly_balance_sheet = pd.DataFrame(financials['quarterly_balance_sheet'])
    quarterly_cash_flow = pd.DataFrame(financials['quarterly_cash_flow'])

    # Clean financial data

    financials_df = pd.concat([quarterly_income_statement, quarterly_balance_sheet, quarterly_cash_flow])
    financials_df.drop_duplicates(inplace=True)
    financials_df = financials_df.transpose()
    financials_df

    # obtain Market Value

    quote = si.get_quote_table(ticker)
    quote['Market Cap']

    market_value = string_to_int(quote['Market Cap'])

    # Create testing dataset

    X_test = pd.DataFrame()
    X_test['current_assets'] = financials_df['totalCurrentAssets']
    X_test['cost_of_goods_sold'] = financials_df['totalOperatingExpenses']
    X_test['depreciation_and_amortization'] = financials_df['depreciation'] + financials_df['intangibleAssets']
    X_test['EBITDA'] = financials_df['grossProfit'] - financials_df['totalOperatingExpenses'] + financials_df['depreciation'] + financials_df['intangibleAssets']
    X_test['inventory'] = financials_df['inventory']
    X_test['net_income'] = financials_df['netIncome']
    X_test['total_receivables'] = financials_df['netReceivables']
    X_test['market_value'] = market_value
    X_test['net_sales'] = financials_df['totalRevenue']
    X_test['total_assets'] = financials_df['totalAssets']
    X_test['total_long_term_debt'] = financials_df['totalLiab']
    X_test['EBIT'] = financials_df['grossProfit'] - financials_df['totalOperatingExpenses']
    X_test['gross_profit'] = financials_df['grossProfit']
    X_test['total_current_liabilities'] = financials_df['totalCurrentLiabilities']
    X_test['retained_earnings'] = financials_df['retainedEarnings']
    X_test['total_revenue'] = financials_df['totalRevenue']
    X_test['total_liabilities'] = financials_df['totalCurrentLiabilities']
    X_test['total_operationg_expenses'] = financials_df['totalOperatingExpenses']

    # get the most-recent and valid column
    X_test.dropna(inplace=True)

    X_scaler = load('interacting_app/scaler.joblib')
    rfc_model = load('interacting_app/randomforestclassifier.joblib')

    X_test_scaled = X_scaler.transform(X_test)
    prediction = rfc_model.predict(X_test_scaled)
    most_recent_result = prediction[0]

    if most_recent_result == 0:
        print(f"According to the most recent financial stats, {ticker} will not bankrupt.")
    elif most_recent_result == 1:
        print(f"According to the most recent financial stats, {ticker} is facing financial difficulty")

        
try:
    run()
except Exception as e:
    print(f"Error: {e}")

c:\Users\yutak\Anaconda3\envs\dev\lib\site-packages\yahoo_fin\stock_info.py:295: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = tables[0].append(tables[1])
c:\Users\yutak\Anaconda3\envs\dev\lib\site-packages\yahoo_fin\stock_info.py:302: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(quote_price)


According to the most recent financial stats, TSLA will not bankrupt.
